# Prerequisites
본 `ipynb` 은 `Python=3.12` 에서 작성하였습니다. Package dependency 를 해결하기 위해 아래 cell 을 실행해주세요.

## Install Python packages

In [ ]:
%pip -q install -U dotenv azure-ai-ml azure-identity azureml-mlflow mlflow

## Load environment variables from a .env file
secret 노출을 피하고 notebook 들간의 일관된 환경변수를 설정하기 위해 `dotenv` 을 이용한다.

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)

AZURE_AML_SUBSCRIPTION_ID = os.getenv("AZURE_AML_SUBSCRIPTION_ID")
AZURE_AML_RESOURCE_GROUP = os.getenv("AZURE_AML_RESOURCE_GROUP")
AZURE_AML_WORKSPACE = os.getenv("AZURE_AML_WORKSPACE")
AZURE_AML_CLUSTER = os.getenv("AZURE_AML_CLUSTER")

# Create a Azure Machine Learning cilent
Azure Machine Learning 의 Client 객체인 `MLClient` 을 생성한다. 본 예제는 Azure CLI 로그인 Credential 을 사용하고 있다. 터미널에서 `az login` 을 정상적으로 완료하여야 한다. `az` 명령어가 설치되어 있지 않다면 [Azure CLI 설치하는 방법](https://learn.microsoft.com/ko-kr/cli/azure/install-azure-cli?view=azure-cli-latest) 을 참고한다.

In [ ]:
from azure.identity import AzureCliCredential
from azure.ai.ml import MLClient

ml_client = MLClient(
    AzureCliCredential(),
    AZURE_AML_SUBSCRIPTION_ID,
    AZURE_AML_RESOURCE_GROUP,
    AZURE_AML_WORKSPACE,
)

# AutoML
AutoML 은 머신러닝 모델의 설계-학습-튜닝 과정을 자동화하여 효율적으로 고품질 모델을 만들 수 있도록 돕는다. AML 에서 AutoML (Automated ML) 을 지원하며 Regression 모델과 tabular 데이터를 통해 이를 실습해보자.

## Define a input dataset
`MLTable` 파일을 이용하여 csv 의 schema 를 정의하고 이를 AML 의 asset 으로 생성한다.

In [ ]:
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes

my_training_data_input = Input(
    type=AssetTypes.MLTABLE, path="./resources/training-mltable-folder",
)

데이터는 어떻게 생겼을까 ?

In [ ]:
import pandas as pd

pd.read_csv("./resources/training-mltable-folder/training-machine-dataset.csv")

## Create a job and submit it
`AutoML` 의 `regression` 기능을 통해 ERP 에 대한 회귀 모델을 학습하는 job 을 생성하여 AML 에 제출한다.

In [ ]:
from azure.ai.ml import automl
from azure.ai.ml.automl import ColumnTransformer

job = automl.regression(
    experiment_name="expr-automl-regression-dpv2",
    training_data=my_training_data_input,
    target_column_name="ERP",
    primary_metric="R2Score",
    n_cross_validations=5,
    enable_model_explainability=True,
    compute=AZURE_AML_CLUSTER,
)
job.set_limits(
    timeout_minutes=600,
    trial_timeout_minutes=20,
    max_trials=5,
    enable_early_termination=True,
)
job.set_featurization(
    mode="custom",
    transformer_params={
        "imputer": [
            ColumnTransformer(
                fields=["CACH"],
                parameters={"strategy": "most_frequent"},
            ),
            ColumnTransformer(
                fields=["PRP"],
                parameters={"strategy": "most_frequent"},
            ),
        ],
    },
    blocked_transformers=["LabelEncoder"],
    column_name_and_types={"CHMIN": "Categorical"},
)
print(f"Created Job: {job.experiment_name}")

returned_job = ml_client.jobs.create_or_update(job)
print(f"Submitted Job: {job.display_name}")

ml_client.jobs.stream(returned_job.name)

## Select the best model from mlflow
`AutoML` 이 여러 알고리즘을 통해 모델을 학습했는데, 그 중에서 성능이 가장 우수한 Best Model 은 어떤 것일까 ?

In [ ]:
import json

import mlflow
from mlflow.tracking.client import MlflowClient
from mlflow.artifacts import download_artifacts

# mlflow SDK tracking uri 를 aml mlflow 로 설정
MLFLOW_TRACKING_URI = ml_client.workspaces.get(
    name=ml_client.workspace_name,
).mlflow_tracking_uri
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
print("\nCurrent tracking uri: {}".format(mlflow.get_tracking_uri()))

# parent run 정보 가져오기
mlflow_client = MlflowClient()
mlflow_parent_run = mlflow_client.get_run(returned_job.name)
print(f"Parent Run: {mlflow_parent_run.info.run_id}")
# Print parent run tags. 'automl_best_child_run_id' tag should be there.
print(f"Parent Run Tags: {json.dumps(mlflow_parent_run.data.tags, indent=2)}")

# best model 찾기
best_child_run_id = mlflow_parent_run.data.tags["automl_best_child_run_id"]
best_run = mlflow_client.get_run(best_child_run_id)
print("Found best child run id: ", best_run.info.run_id)


## (Optional) Download the best model locally
모델은 mlflow 에 artifact 로 저장된다. 사용자는 mlflow 의 tracking uri 을 AML 쪽으로 변경하여 mlflow SDK 를 통해 artifact 에 접근할 수 있다.

In [ ]:
import os

# artifact 다운로드를 위한 로컬 디렉토리 생성 후 다운로드
local_dir = "./artifact_downloads"
if not os.path.exists(local_dir):
    os.mkdir(local_dir)
local_path = download_artifacts(
    run_id=best_run.info.run_id, artifact_path="outputs", dst_path=local_dir
)

# 폴더내 mlflow 모델 파일들 확인
os.listdir(local_dir + "/outputs/mlflow-model")

# Model Deployment
모델을 학습했으니, 마지막으로 모델 배포다. 모델 배포 모듈은 Endpoint 와 Deployment 로 구성된다. Endpoint 는 model deployment 에 대한 reverse proxy 로, 권한&인증/traffic 관리/모니터링 등의 기능을 가진다. Deployment 는 모델을 안정적으로 배포하는 환경을 구축하며, 모델에 따른 다양한 runtime 환경을 지원한다.

## Create a managed online endpoint
먼저 endpoint 를 배포해보자. 실시간 추론을 지원하는 online type 으로 배포한다.

In [ ]:
import datetime
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    ProbeSettings,
)

endpoint_name = "regression-" + datetime.datetime.now().strftime("%m%d%H%M%f")
endpoint = ManagedOnlineEndpoint(name=endpoint_name, auth_mode="key")
ml_client.begin_create_or_update(endpoint).result()

## Create a online deployment
다음은 Deployment 이다. Endpoint 와 동일하게 실시간 추론 타입으로 배포하자.

In [ ]:
model = Model(
    path=f"azureml://jobs/{best_run.info.run_id}/outputs/artifacts/outputs/mlflow-model/",
    name="hardware-performance-model",
    type=AssetTypes.MLFLOW_MODEL,
)
registered_model = ml_client.models.create_or_update(model)
registered_model.id

deployment = ManagedOnlineDeployment(
    name="hardware-performance-deploy",
    endpoint_name=endpoint_name,
    model=registered_model.id,
    instance_type="Standard_DS3_V2",
    instance_count=1,
    liveness_probe=ProbeSettings(
        failure_threshold=30,
        success_threshold=1,
        timeout=2,
        period=10,
        initial_delay=2000,
    ),
    readiness_probe=ProbeSettings(
        failure_threshold=10,
        success_threshold=1,
        timeout=10,
        period=10,
        initial_delay=2000,
    ),
)
ml_client.online_deployments.begin_create_or_update(deployment).result()

# 생성된 deployment 에 트래픽 할당
endpoint.traffic = {"hardware-performance-deploy": 100}
ml_client.begin_create_or_update(endpoint).result()

## Test Inference
배포가 완료되었으니, 테스트해볼까요 ?

In [ ]:
import pandas as pd

# Test 파일 생성
data = pd.read_csv("./resources/training-mltable-folder/training-machine-dataset.csv")
data = data.drop("ERP", axis=1)
data_json = data.to_json(orient="records", indent=4)

request_file_name = "request.json"
with open(request_file_name, "w") as request_file:
    request_file.write('{"input_data": {"data": '+ data_json + "}}")

# Endpoint 에 추론 요청
response = ml_client.online_endpoints.invoke(
    endpoint_name="regression-10252057277986",
    deployment_name="hardware-performance-deploy",
    request_file=request_file_name,
)
response

## (Optional) Delete all resources
필요하다면 삭제하자.

In [ ]:
ml_client.online_endpoints.begin_delete(name=endpoint_name)